# Vanilla RNN & Glove Embeddings

**CNNs and Multi Layer Perceptrons are feed-forward neural networks. They do not adequately model sequences.**

A sequence is an ordered collection of items. Traditional machine learning assumes data points to be independently and identically distributed (IID), but in many situations, like with language, speech, and time-series data, one data item
depends on the items that precede or follow it. Such data is also called sequence data. Sequential information is everywhere in human language. For example, speech can be considered a sequence of basic units called phonemes. 
In a language like English, words in a sentence are not haphazard. They might be constrained by the words that come before or after them. For example, in the English language, the preposition “of” is likely followed by the article “the”; for
example, “The lion is the king of the jungle.” As another example, in many languages, including English, the number of a verb must agree with the number of the subject in a sentence.Sometimes these dependencies or constraints can be arbitrarily long.
In short, **understanding sequences is essential to understanding human language.**

We begin by introducing the most basic neural network sequence model: the **recurrent neural network (RNN)**.
There are several different members in the RNN family, but here we work with only **the most basic form, sometimes called the Elman RNN or Vanilla RNN**. 

The goal of recurrent networks — both the basic Elman form and the more complicated forms — is to learn a representation of a sequence. This is done by maintaining a hidden state vector that captures the current state of the sequence.
The hidden state vector is computed from both a current input vector and the previous hidden state vector. **The
hidden vector of each time step is dependent on both the input at that time step and the hidden vector from the previous time step**.

**A new hidden vector is computed using a hidden-to-hidden weight matrix to map the previous hidden state vector and an input-to-hidden weight matrix to map the input vector. Crucially, the hidden-to-hidden and input-to-hidden weights are shared across
the different time steps. The intuition you should take away from this fact is that, during training, these weights will be adjusted so that the RNN is learning how to incorporate incoming information and maintain a state representation summarizing the input seen so far. The RNN does not have any way of knowing which time step it is on. Instead, it is simply learning how to transition from one time step to another and maintain a state representation that will minimize its loss function**.

Using the same weights to transform inputs into outputs at every time step is another example of parameter sharing. We saw how CNNs share parameters across space. CNNs use parameters, called kernels, to compute outputs from subregions in the input data. You can think of an RNN sharing parameters across time and a CNN sharing parameters across space.

Because words and sentences can be of different lengths, the RNN or any sequence model should be equipped to handle variable-length sequences. One possible technique is to restrict sequences to a fixed length artificially. Another technique is masking; in brief, **masking allows for the data to signal when certain inputs should not count toward the gradient or the eventual output**.


**Word Vectors** are often used as a fundamental component for downstream NLP tasks, e.g. question answering, text generation, translation, etc., so it is important to build some intuitions as to their strengths and weaknesses. Here, you will explore word vectors derived via **GloVe**.

## Import Packages

In [1]:
import os                                 # to create 'serialised' directory
import pandas as pd
import numpy as np
import json                               # for serialising dictionaries
import pickle                             # for serialising numpy arrays
import re
import time
import random
from collections import Counter           # this is for stopwords removal
from nltk.corpus import stopwords         # this is for stopwords removal

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split

import spacy                              # for tokenisation

In [2]:
pd.set_option('display.max_row', None)              # show all rows of a dataframe
pd.set_option('display.max_column', None)           # show all columns of a dataframe
pd.set_option('display.max_colwidth', None)         # show the full width of columns
pd.set_option('precision', 2)                       # round to 2 decimal points
pd.options.display.float_format = '{:,.2f}'.format  # comma separators and two decimal points: 4756.7890 => 4,756.79 and 4656 => 4,656.00 

In [3]:
np.random.seed(0)           # global seeding. Sklearn uses this internally therefore there is no need to set a random seed when using Sklearn
random.seed(0)

In [4]:
TOK = spacy.load('en_core_web_sm')                                         # for tokenisation
stop_words = stopwords.words('english')                                    # for stopwords
stopwords_dict = Counter(stop_words)                                       # for stopwords
max_len = 100                                                              # for encoding

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')      # pytorch cuda

C:\Users\Mari\miniconda3\envs\summer\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


## Functions

### 1. Read Datasets

In [6]:
def read_csv(filepath, filename, encoding='utf-8'):
    full_path = '{}\{}'.format(filepath, filename)
    return pd.read_csv(full_path, encoding=encoding, header=None)

### 2. Data Cleaning

In [7]:
def clean_tokenise(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    text = ' '.join([word for word in text.split() if word not in stopwords_dict])  # remove stopwords
    
    return [token.text for token in TOK.tokenizer(text)]

### 3. Serialisation / Deserialisation

https://www.tutorialspoint.com/object_oriented_python/object_oriented_python_serialization.htm

In the context of data storage, serialization is the process of translating data structures or object state into a format that can be stored (for example, in a file or memory buffer) or transmitted and reconstructed later.

In serialization, an object is transformed into a format that can be stored, so as to be able to deserialize it later and recreate the original object from the serialized format.

We can do this with Pickle, JSON, ...

In [8]:
def serialise(obj_type, filepath, filename, obj):
    full_path = '{}\{}'.format(filepath, filename)
    if obj_type == 'python':
        with open(full_path, 'w') as f:
            json.dump(obj, f)
            
    if obj_type in ['numpy', 'pandas', 'tensor']:
        with open(full_path, 'wb') as f:
            pickle.dump(obj, f)                                         # protocol 0 is printable ASCII
        
        

def deserialise(obj_type, filepath, filename):
    full_path = '{}\{}'.format(filepath, filename)
    if obj_type == 'python':
        with open(full_path, 'r') as f:
            return json.load(f)
            
    if obj_type in ['numpy', 'pandas', 'tensor']:
        with open(full_path, 'rb') as f:
            return pickle.load(f)                                        # protocol 0 is printable ASCII

### 4. Vocabulary from the Traing Corpus

In [9]:
def tokens_dict(series):
    tokens = series.explode()
    tokens = tokens.tolist()
    tokens = Counter(tokens)                                    # frequency distribution

    num_words_before = len(tokens.keys())
    
    #To avoid 'RuntimeError: dictionary changed size during iteration' error, we need to make a .copy() of the dictionary.
    #This way we iterate over the original dictionary keys and delete elements on the fly.
    for k,v in tokens.copy().items():
        if v < 2:
            del tokens[k]

    tokens = {word: i for i,word in enumerate(tokens.keys())}    # word2index dictionary

    tokens['<UNK>'] = len(tokens)
    tokens['<MASK>'] = len(tokens)
    tokens['<END>'] = len(tokens)
    tokens['<START>'] = len(tokens)
    tokens[''] = len(tokens)
    
    num_words_after = len(tokens.keys())
    
    return num_words_before, num_words_after, tokens

### 5. All Glove Embeddings

In [10]:
def load_glove_from_file(glove_filepath):
    
    embeddings_dict = {} 
    
    with open(glove_filepath, mode='r', encoding="utf-8") as f:
        for index,line in enumerate(f):
            line_split = line.split()
            word = line_split[0]
            embeddings_dict[word] = np.array(line_split[1:], 'float32')
    
    return embeddings_dict

### 6. Embedding Matrix

In [11]:
def make_embedding_matrix(embeddings_dict, tokens):  
    
    embedding_size = len(next(iter(embeddings_dict.values())))     # length of each word embedding (e.g.300 dimensions)
    
    embeddings_matrix = np.zeros((len(tokens)+1, embedding_size))  # len(tokens) is the length of the vocabulary (i.e. unique words in the corpus)
                                                                   # the '+1' after len(tokens) is necessary because we want the first embedding (i.e. at index 0) to be a zero vector; this will be used for the paddings (we want all the '0' paddings to be paired with the zero vector)
    not_in_glove = []
    for token,i in tokens.items():
        if token in embeddings_dict:
            embeddings_matrix[i+1, :] = embeddings_dict[token]     # '+1' :see comment above
        else:
            not_in_glove.append(token)
            embedding_i = torch.ones(1, embedding_size)
            torch.nn.init.xavier_uniform_(embedding_i)             # if word is not in Glove emebeddings, creates a new vector with random numbers drawn from pytorch xavier_uniform distribution
            embeddings_matrix[i+1, :] = embedding_i                # '+1' :see comment above

    return not_in_glove, embeddings_matrix

### 7. Encoding

In [12]:
# def encoding(input_sequence, tokens, max_len):
#     encoded = np.zeros(max_len, dtype=int)
#     encoded_lst = np.array([tokens.get(word, tokens['<UNK>']) for word in input_sequence])
#     length = min(max_len, len(encoded_lst))
#     encoded[:length] = encoded_lst[:length]
#     return encoded

def encoding(input_sequence):
    encoded = np.zeros(max_len, dtype=int)
    encoded_lst = np.array([tokens.get(word, tokens['<UNK>']) for word in input_sequence])
    length = min(max_len, len(encoded_lst))
    encoded[:length] = encoded_lst[:length]
    return encoded

### 8. Vanilla RNN Classifier

In [63]:
class RNNClassifier(nn.Module):
    def __init__(self, embedding_size, num_embeddings,                      # embedding_size = 300, num_embeddings=95465
                 rnn_hidden_size, output_dim, dropout_p, batch_first=True,
                 pretrained_embeddings=None, padding_idx=0):                # pretrained_embeddings = embedding_matrix
                                                                            # padding_idx=0 makes sure that the padding vector (which in our embedding matrix is at index 0) doesn't get updated during training when Freeze=False
        super(RNNClassifier, self).__init__()                      

        if pretrained_embeddings is None:

            self.emb = nn.Embedding(embedding_dim=embedding_size,
                                    num_embeddings=num_embeddings,
                                    padding_idx=padding_idx)        
        else:
            pretrained_embeddings = torch.from_numpy(pretrained_embeddings).float()
            # freeze=False : the tensor does not get updated in the learning process. Equivalent to self.emb.weight.requires_grad = False
            self.emb = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=True, padding_idx=padding_idx)  
            
#             self.emb = nn.Embedding(embedding_dim=embedding_size,
#                                     num_embeddings=num_embeddings,
#                                     padding_idx=padding_idx,
#                                     _weight=pretrained_embeddings)
            
            
            # nn.Embedding is a model parameter layer, which is by default trainable.
            # If you want to fine-tune word vectors during training, these word vectors are treated as model parameters 
            # and are updated by backpropagation. You can also make it untrainable by freezing its gradient 
            # (False ==> freezes the backprop) 
#             self.emb.weight.requires_grad=False    
            
        self.rnn = nn.RNN(input_size=embedding_size, hidden_size=rnn_hidden_size, batch_first=batch_first) #initialise RNN model
        
        self._dropout_p = dropout_p      
        self.fc1 = nn.Linear(in_features=rnn_hidden_size, out_features=rnn_hidden_size)
        self.fc2 = nn.Linear(in_features=rnn_hidden_size, out_features=output_dim)
        self.sigmoid = nn.Sigmoid()
        

    def forward(self, x_in, apply_sigmoid=False):
    # Note that we don't use the Sigmoid activation in our final layer during training because we use the 
    # nn.BCEWithLogitsLoss() loss function which automatically applies the the Sigmoid activation.
        x_embedded = self.emb(x_in.long())                # x_in is the encoded review e.g. [3386  603 1112    0    0    0    0    0    0    0]. It comes from the train_loader
        hiddens = self.rnn(x_embedded)                    # the output of self.rnn is a tuple
        hiddens = hiddens[0]                              # we need the first item of the tuple (i.e. index 0)
        hiddens = hiddens[:,-1,:]                         # for each batch item, we need the last hidden state (second dimension with value -1)
        
        hidden_layer = F.relu(F.dropout(self.fc1(hiddens), p=self._dropout_p))
        output_layer = self.fc2(hidden_layer).squeeze(1)   # squeeze(1) to make the prediction the same shape as the target i.e. as a scaler not a 2d-vector

        if apply_sigmoid: 
            output_layer = sef.sigmoid(output_layer)
        return output_layer

### 9.  Accuracy

In [14]:
def binary_acc(y_hat, y):
    y_hat_label = torch.round(torch.sigmoid(y_hat))

    correct_predictions_sum = (y_hat_label == y).sum().float()
    acc = correct_predictions_sum/y.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

### 10. Parameters

In [15]:
def params(full_embedding_layer_name):
    for param in full_embedding_layer_name.parameters():
        return param


def check_params(classifier_name):
    for name, child in classifier_name.named_children():
        print('Layer name: {} --- {}'.format(name, child), end='\n\n')            
        print('ToT Params: {:,}'. format(sum(p.numel() for p in child.parameters())), end='\n\n') 
    
        count = 0
        for param in child.parameters():
            print('Param length: {:,}'.format(len(param)), end='\n\n')
            print(param, end='\n\n')
            print('Are parameters being updated during backprop? {}'.format(param.requires_grad), end='\n\n')
            count += 1

        print('Total Sets of Parameters: {}'.format(count), end='\n\n')
        print('*' * 90)
    

def num_params(classifier_name):
    
    # PyTorch torch.numel() method returns the total number of elements in the input tensor
    trainable_parameters = sum(param.numel() for param in classifier_name.parameters() if param.requires_grad)  
    all_parameters = sum(param.numel() for param in classifier_name.parameters())  
    
    return trainable_parameters, all_parameters

### 11. Inference on new data

In [16]:
class Pipeline:
    def __init__(self):
        self.tasks = []
        
    def task(self, depends_on=None):
        idx = 0
        if depends_on:
            idx = self.tasks.index(depends_on) + 1
        def inner(f):
            self.tasks.insert(idx, f)
            return f
        return inner
    
    # Add the run() method which should take in an 'input_' argument
    def run(self, input_):
        output = input_
        # Iterate through the self.tasks property, and call each function with the previous output
        for task in self.tasks:
            output = task(output)
        return output

In [17]:
inference_pipeline = Pipeline()

In [18]:
inference_pipeline.tasks = [clean_tokenise, encoding]   #calls the functions created at the top of the notebook

In [19]:
# add new functions to the pipeline

@inference_pipeline.task(depends_on=encoding)
def convert_to_tensor(text):
    return torch.Tensor(text)

@inference_pipeline.task(depends_on=convert_to_tensor)
def infer(text):
    # Disable grad
    with torch.no_grad():
        
        # Generate prediction
        prediction = classifier(text.unsqueeze(0))
        probability_value = classifier.sigmoid(prediction).item()
        
        if probability_value < 0.5:
            prediction_label = 'Negative'
        else:
            prediction_label = 'Positive'
            
    return prediction_label, probability_value

## Code

### Directories

In [20]:
dataset_directory = r'C:\Users\Mari\Desktop\MACHINE_LEARNING\NLP_Stanford_University\BOOK\YELP\dataset'
train_set_filename = 'raw_train.csv'
test_set_filename = 'raw_test.csv'

serialise_directory = r'C:\Users\Mari\Desktop\MACHINE_LEARNING\NLP_Stanford_University\BOOK\YELP\serialised'

In [ ]:
# Create the 'serialised' directory if it doesn't exist

try:
    os.makedirs(serialise_directory)
except FileExistsError:
    # directory already exists
    pass

### Read Datasets

In [ ]:
# Import Datasets
train_original = read_csv(dataset_directory, train_set_filename)
test = read_csv(dataset_directory, test_set_filename)

# Add column names
train_original.columns = ['target', 'review']
test.columns = ['target', 'review']

# Split Targets from Features
X_train_original = train_original['review']
y_train_original = train_original['target']
X_test = test['review']
y_test = test['target']

# Re-label Target: In PyTorch labels need to start at 0
# 1 ==> 0 (these are negative reviews)
# 2 ==> 1 (these are positive reviews)
y_train_original = y_train_original - 1
y_test = y_test - 1

# Split 'train_original' in 'train' and 'val' 
X_train, X_val, y_train, y_val = train_test_split(X_train_original, y_train_original, test_size=0.3)

In [ ]:
# print(type(X_train), X_train.shape)
# print(type(X_val), X_val.shape)
# print(type(X_test), X_test.shape)
# print(type(y_train), y_train.shape)
# print(type(y_val), y_val.shape)
# print(type(y_test), y_test.shape, end='\n\n')
# print(X_train.head(2), end='\n\n')
# print(y_train.head(2))

### Data Cleaning And Tokenization

In addition to creating a subset that has three partitions for training, validation, and testing, we also minimally clean the data by adding whitespace around punctuation symbols and removing extraneous symbols that aren’t punctuation for all the splits.


1. **apply** works on a row / column basis of a DataFrame 
2. **applymap** works element-wise on a DataFrame
3. **map** works element-wise on a Series

In [ ]:
X_train = X_train.map(clean_tokenise)
X_val = X_val.map(clean_tokenise)
X_test = X_test.map(clean_tokenise)

### Glove

In [ ]:
# From glove txt, create a dictionary of all glove embeddings where KEY is a WORD, and VALUE is a NUMPY ARRAY:
glove_embeddings = load_glove_from_file('C:/GloVe/glove.6B.300d.txt')

### Create Vocabulary From Training Corpus

The embedding matrix (see later) is created only from the training dataset.

The training dataset should be sufficiently rich/representative enough to cover all data you expect to see in the future.

New data must have the same integer encoding as the training data prior to being mapped onto the embedding when making a prediction.

In [ ]:
# create a DICT of the unique words in the training set
num_words_before, num_words_after, tokens = tokens_dict(X_train)

### Create Embedding Matrix

The embedding is created from the training dataset.

It should be sufficiently rich/representative enough to cover all data you expect to in the future.

New data must have the same integer encoding as the training data prior to being mapped onto the embedding when making a prediction.

In [ ]:
not_in_glove, embedding_matrix = make_embedding_matrix(glove_embeddings, tokens)

### Encoding Training Dataset

We need to convert our text into a numerical form that can be fed to our model as input.

1. We have create a vocabulary (see section '10. Vocabulary') where each key is a unique word from the training corpus, and each value is the index of that word in the 'tokens' dictionary.
2. Choose the maximum length of any review.
3. Encode each list of tokens by replacing each word with its index from the 'tokens' dictionary.

Note: **mean_len** (see below) is the mean of tokens length in the training set. We set the max length of the encoded reviews equal to the mean_len.

In [ ]:
X_train_encoded = X_train.map(lambda input_lst: encoding(input_lst, tokens=tokens, max_len=max_len))   
X_val_encoded = X_val.map(lambda input_lst: encoding(input_lst, tokens=tokens, max_len=max_len))
X_test_encoded = X_test.map(lambda input_lst: encoding(input_lst, tokens=tokens, max_len=max_len))

### PyTorch Dataset

In [ ]:
# Convert pd.Series to PyTorch Tensors
# NB: set the values in X_train, X_val and X_test as a list of arrays (as opposed to array of arrays) --- see above

x_train_tensor = torch.Tensor(list(X_train_encoded.values))
x_val_tensor = torch.Tensor(list(X_val_encoded.values))
x_test_tensor = torch.Tensor(list(X_test_encoded.values))
y_train_tensor = torch.Tensor(list(y_train.values))
y_val_tensor = torch.Tensor(list(y_val.values))
y_test_tensor = torch.Tensor(list(y_test.values))

In [ ]:
# Create a full dataset (like a DataFrame in Pandas) from the two tensors
train_dataset =  TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

### Serialisation

In [ ]:
# Serialise datasets after tokenisation
serialise('pandas', serialise_directory, 'X_train', X_train)
serialise('pandas', serialise_directory, 'X_val', X_val)
serialise('pandas', serialise_directory, 'X_test', X_test)
serialise('pandas', serialise_directory, 'y_train', y_train)
serialise('pandas', serialise_directory, 'y_val', y_val)
serialise('pandas', serialise_directory, 'y_test', y_test)
serialise('pandas', serialise_directory, 'mean_len', mean_len)

In [ ]:
# Serialise glove embeddings (althought 'glove_embeddings' is a dictionary, its values are numpy arrays therefore we need to choose 'numpy')
serialise('numpy', serialise_directory, 'glove_embeddings', glove_embeddings)

In [ ]:
# Serialise tokens
serialise('python', serialise_directory, 'tokens', tokens)
serialise('python', serialise_directory, 'num_words_before', num_words_before)
serialise('python', serialise_directory, 'num_words_after', num_words_after)

In [ ]:
# Serialise datasets embedding matrix
serialise('numpy', serialise_directory, 'embedding_matrix', embedding_matrix)
serialise('python', serialise_directory, 'not_in_glove', not_in_glove)

In [ ]:
# Serialise encoded datasets
serialise('pandas', serialise_directory, 'X_train_encoded', X_train_encoded)
serialise('pandas', serialise_directory, 'X_val_encoded', X_val_encoded)
serialise('pandas', serialise_directory, 'X_test_encoded', X_test_encoded)

In [ ]:
# Serialise tensors
serialise('tensor', serialise_directory, 'x_train_tensor', x_train_tensor)
serialise('tensor', serialise_directory, 'x_val_tensor', x_val_tensor)
serialise('tensor', serialise_directory, 'x_test_tensor', x_test_tensor)
serialise('tensor', serialise_directory, 'y_train_tensor', y_train_tensor)
serialise('tensor', serialise_directory, 'y_val_tensor', y_val_tensor)
serialise('tensor', serialise_directory, 'y_test_tensor', y_test_tensor)

In [ ]:
# Serialise PyTorch Dataset
serialise('tensor', serialise_directory, 'train_dataset', train_dataset)
serialise('tensor', serialise_directory, 'val_dataset', val_dataset)
serialise('tensor', serialise_directory, 'test_dataset', test_dataset)

### Deserialisation

In [ ]:
# Deserialise datasets after tokenisation
X_train = deserialise('pandas', serialise_directory, 'X_train')
X_val = deserialise('pandas', serialise_directory, 'X_val')
X_test = deserialise('pandas', serialise_directory, 'X_test')

In [21]:
y_train = deserialise('pandas', serialise_directory, 'y_train')
y_val = deserialise('pandas', serialise_directory, 'y_val')
y_test = deserialise('pandas', serialise_directory, 'y_test')

In [22]:
# print(type(X_train), X_train.shape)
# print(type(X_val), X_val.shape)
# print(type(X_test), X_test.shape)
# print(type(y_train), y_train.shape)
# print(type(y_val), y_val.shape)
# print(type(y_test), y_test.shape, end='\n\n')
# print(X_train.head(2), end='\n\n')
# print(y_train.head(2), end='\n\n')
# print(type(mean_len))
# print(mean_len)

In [23]:
# Deserialise glove embeddings
glove_embeddings = deserialise('numpy', serialise_directory, 'glove_embeddings')

In [24]:
# print(type(glove_embeddings))
# print(len(glove_embeddings))
# print(glove_embeddings['car'])

In [25]:
# Deserialise datasets
tokens = deserialise('python', serialise_directory, 'tokens')
num_words_before = deserialise('python', serialise_directory, 'num_words_before')
num_words_after = deserialise('python', serialise_directory, 'num_words_after')

In [26]:
# print(type(tokens))
# print(len(tokens))
# print(tokens['car'], end='\n\n')
# print(type(num_words_before))
# print(num_words_before, end='\n\n')
# print(type(num_words_after))
# print(num_words_after, end='\n\n')

In [27]:
# Deserialise embedding matrix
embedding_matrix = deserialise('numpy', serialise_directory, 'embedding_matrix')
not_in_glove = deserialise('python', serialise_directory, 'not_in_glove')

In [28]:
# print(type(embedding_matrix))
# print(embedding_matrix.shape)
# print(embedding_matrix[56], end='\n\n')
# print(type(not_in_glove))
# print(len(not_in_glove))
# print(not_in_glove[:15])

In [29]:
# Deserialise encoded datasets
X_train_encoded = deserialise('numpy', serialise_directory, 'X_train_encoded')
X_val_encoded = deserialise('numpy', serialise_directory, 'X_val_encoded')
X_test_encoded = deserialise('numpy', serialise_directory, 'X_test_encoded')

In [30]:
# print(type(X_train_encoded))
# print(X_train_encoded.shape)
# print(type(X_val_encoded))
# print(X_val_encoded.shape)
# print(type(X_test_encoded))
# print(X_test_encoded.shape, end='\n\n')
# print(X_train.sample(2), end='\n\n')
# print((X_train_encoded.map(lambda x: len(x))).mean())
# print((X_val_encoded.map(lambda x: len(x))).mean())
# print((X_test_encoded.map(lambda x: len(x))).mean())

In [31]:
# Deserialise tensors
x_train_tensor = deserialise('tensor', serialise_directory, 'x_train_tensor')
x_val_tensor = deserialise('tensor', serialise_directory, 'x_val_tensor')
x_test_tensor = deserialise('tensor', serialise_directory, 'x_test_tensor')
y_train_tensor = deserialise('tensor', serialise_directory, 'y_train_tensor')
y_val_tensor = deserialise('tensor', serialise_directory, 'y_val_tensor')
y_test_tensor = deserialise('tensor', serialise_directory, 'y_test_tensor')

In [32]:
# print(type(x_train_tensor), x_train_tensor.shape)
# print(type(x_val_tensor), x_val_tensor.shape)
# print(type(x_test_tensor), x_test_tensor.shape)
# print(type(y_train_tensor), y_train_tensor.shape)
# print(type(y_val_tensor), y_val_tensor.shape)
# print(type(y_test_tensor), y_test_tensor.shape)

In [33]:
# Deserialise PyTorch Dataset
train_dataset = deserialise('tensor', serialise_directory, 'train_dataset')
val_dataset = deserialise('tensor', serialise_directory, 'val_dataset')
test_dataset = deserialise('tensor', serialise_directory, 'test_dataset')

In [34]:
# print(type(train_dataset), len(train_dataset))
# print(type(val_dataset), len(val_dataset))
# print(type(test_dataset), len(test_dataset))

### PyTorch DataLoader

In [39]:
# For small dataset is fine to use the whole training data at every training step (i.e. batch gradient descent). 
# If we want to go serious about all this, we must use mini-batch gradient descent. Thus, we need mini-batches. 
# Thus, we need to slice our dataset accordingly. Do you want to do it manually?! Me neither!
# So we use the 'DataLoader' class for this job. We tell it which dataset to use, the desired mini-batch size and if we’d 
# like to shuffle it or not. That’s it!
# Our loader will behave like an iterator, so we can loop over it and fetch a different mini-batch every time.

train_loader = DataLoader(dataset=train_dataset, batch_size=1048, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=1048, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=1048, shuffle=False)

# To retrieve a sample mini-batch, one can simply run the command below.
# It will return a list containing two tensors: one for the features, another one for the labels:
# next(iter(train_loader))

### Initialise Classifier

At its core, the training routine is responsible for instantiating the model, iterating over the dataset, computing the output of the model when given the data as input, computing the loss (how wrong the model is), and updating the model proportional to the loss. 

Although this may seem like a lot of details to manage, there are not many places to change the training routine, and as such it will become habitual in your deep learning development process.

In [64]:
num_embeddings, embedding_dim = embedding_matrix.shape
rnn_hidden_size = 64
output_dim = 1
dropout_p = 0.3

In [65]:
# Initialise classifier
# We need to send our model to the same device where the data is. If our data is made of GPU tensors, 
# our model must “live” inside the GPU as well. That's what '.to(device)' is there for.

classifier = RNNClassifier(embedding_size=embedding_dim, num_embeddings=num_embeddings, batch_first=True,
                           rnn_hidden_size=rnn_hidden_size, output_dim=output_dim,
                           dropout_p=dropout_p, pretrained_embeddings=embedding_matrix, padding_idx=0).to(device)

In [66]:
# Loss and optimizer
loss_func = nn.BCEWithLogitsLoss()

In [67]:
# Optimizer
optimizer = optim.Adam(classifier.parameters(), lr=0.001)
#optimizer = optim.Adam(filter(lambda p: p.requires_grad, classifier.parameters()), lr=0.01)    #filtering only for the params that require updating doesn't speed up training

### Serialise / Deserialise Embedding Parameters before training

In [68]:
params_before = params(classifier.emb)

In [69]:
# Serialise embedding params before training
serialise('tensor', serialise_directory, 'params_before', params_before)

In [70]:
# Deserialise embedding params before training
params_before = deserialise('tensor', serialise_directory, 'params_before')

In [71]:
params_before

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2319, -0.1954,  0.0334,  ...,  0.1139,  0.1851, -0.3686],
        [ 0.0206,  0.3291, -0.0162,  ..., -0.0801, -0.2023,  0.0679],
        ...,
        [-0.1062,  0.1081,  0.0919,  ..., -0.0722,  0.0423, -0.0406],
        [-0.0733,  0.1390, -0.0326,  ...,  0.1367,  0.0150, -0.0859],
        [-0.1353, -0.1157,  0.0118,  ..., -0.0008,  0.0778, -0.0832]])

In [72]:
# check params initialised by the classifier
check_params(classifier)

Layer name: emb --- Embedding(95399, 300, padding_idx=0)

ToT Params: 28,619,700

Param length: 95,399

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2319, -0.1954,  0.0334,  ...,  0.1139,  0.1851, -0.3686],
        [ 0.0206,  0.3291, -0.0162,  ..., -0.0801, -0.2023,  0.0679],
        ...,
        [-0.1062,  0.1081,  0.0919,  ..., -0.0722,  0.0423, -0.0406],
        [-0.0733,  0.1390, -0.0326,  ...,  0.1367,  0.0150, -0.0859],
        [-0.1353, -0.1157,  0.0118,  ..., -0.0008,  0.0778, -0.0832]])

Are parameters being updated during backprop? False

Total Sets of Parameters: 1

******************************************************************************************
Layer name: rnn --- RNN(300, 64, batch_first=True)

ToT Params: 23,424

Param length: 64

Parameter containing:
tensor([[-0.0342,  0.0777,  0.0987,  ..., -0.1055, -0.0512,  0.0473],
        [ 0.0541,  0.1067,  0.0610,  ...,  0.0290, -0.1078,  0.0907],
        [ 0.06

In [73]:
# trainable / all params before training
trainable_params, all_params = num_params(classifier)
print('The model has {:,} trainable parameters'.format(trainable_params))
print('The model has {:,} parameters overall'.format(all_params))

The model has 27,649 trainable parameters
The model has 28,647,349 parameters overall


## Training loop

The training loop is composed of two loops: an inner loop over minibatches in the dataset, and an outer loop, which repeats the inner loop a number of times. In the inner loop, losses are computed for each minibatch, and the optimizer is used to
update the model parameters.

In [74]:
start = time.time()
n_epochs = 10
n_epoch_freezed = 10

print('Starting training', end='\n\n')


# Enumerate epochs
epoch = 0

# For a certain number of epochs (defined by 'n_epoch_freezed'), the emebdding matrix is frozen, then it is unfrozen 
# i.e. the embeddings get trained (except for the padding vector which remains 0)
for epoch in range(n_epochs):
    if epoch < n_epoch_freezed:   
        pass   # keep the embedding layer frozen (i.e. classifier.emb.weight.requires_grad=False as set in section 8 above)
    else: 
        classifier.emb.weight.requires_grad=True

    # Training part
    classifier.train()
    
    epoch_train_loss = 0
    epoch_train_acc = 0
    
    for i, (x_train, y_train) in enumerate(train_loader):
        x_train = x_train.to(device)
        y_train = y_train.to(device)

        # Clear the gradients
        optimizer.zero_grad()
        
        # Forward propagation: compute the model output (i.e. predictions)
        y_pred = classifier(x_in=x_train)

        #print(x_train.requires_grad, y_train.requires_grad, y_pred.requires_grad)
                                                                                                        
        # Loss calculation
        t_loss = loss_func(y_pred, y_train)
        
        # Accuracy
        t_acc = binary_acc(y_pred, y_train)
        
        # Backward propagation: use loss to produce gradients
        t_loss.backward()
        
        # Weight optimization: use optimizer to take gradient step and update parameters (w,b) 
        optimizer.step()
        
        epoch_train_loss += t_loss.item()
        epoch_train_acc += t_acc.item()
                                                                                                             
   
    # Evaluation part
    classifier.eval() # .eval() tells PyTorch that we do not want to perform back-propagation during inference
    
    epoch_val_loss = 0
    epoch_val_acc = 0
    
    #We use torch.no_grad() which reduces memory usage and speeds up computation.
    with torch.no_grad():     #https://discuss.pytorch.org/t/model-eval-vs-with-torch-no-grad/19615/3 : torch.no_grad() deals with the autograd engine and stops it from calculating the gradients, which is the recommended way of doing validation
        for i, (x_val, y_val) in enumerate(val_loader):
            x_val = x_val.to(device)
            y_val = y_val.to(device)
        
            # Forward propagation: compute the model output (i.e. predictions)
            y_pred = classifier(x_in=x_val)     #tensors of probabilities
        
            # Loss calculation
            v_loss = loss_func(y_pred, y_val)  
            
            # Accuracy
            v_acc = binary_acc(y_pred, y_val)
            
            epoch_val_loss += v_loss.item()
            epoch_val_acc += v_acc.item()
            
    print('Epoch: {} | Train Loss: {:.3f} | Val Loss: {:.3f} | Train Acc: {:.3f} | Val Acc: {:.3f}'.format(epoch,
                                                                                epoch_train_loss/len(train_loader),
                                                                                epoch_val_loss/len(val_loader),
                                                                                epoch_train_acc/len(train_loader),
                                                                                epoch_val_acc/len(val_loader)))
    
    print(num_params(classifier), end='\n\n')
    
epoch += 1
    

print()
print('Training complete', end='\n\n')

end = time.time()
print(end - start)

Starting training

Epoch: 0 | Train Loss: 0.684 | Val Loss: 0.687 | Train Acc: 55.355 | Val Acc: 54.634
(27649, 28647349)

Epoch: 1 | Train Loss: 0.686 | Val Loss: 0.683 | Train Acc: 54.923 | Val Acc: 55.292
(27649, 28647349)

Epoch: 2 | Train Loss: 0.683 | Val Loss: 0.687 | Train Acc: 55.235 | Val Acc: 54.447
(27649, 28647349)

Epoch: 3 | Train Loss: 0.685 | Val Loss: 0.686 | Train Acc: 54.960 | Val Acc: 54.727
(27649, 28647349)

Epoch: 4 | Train Loss: 0.684 | Val Loss: 0.688 | Train Acc: 54.928 | Val Acc: 54.373
(27649, 28647349)

Epoch: 5 | Train Loss: 0.686 | Val Loss: 0.686 | Train Acc: 54.680 | Val Acc: 54.720
(27649, 28647349)

Epoch: 6 | Train Loss: 0.684 | Val Loss: 0.684 | Train Acc: 55.163 | Val Acc: 55.093
(27649, 28647349)

Epoch: 7 | Train Loss: 0.686 | Val Loss: 0.687 | Train Acc: 54.872 | Val Acc: 54.416
(27649, 28647349)

Epoch: 8 | Train Loss: 0.685 | Val Loss: 0.684 | Train Acc: 54.920 | Val Acc: 55.000
(27649, 28647349)

Epoch: 9 | Train Loss: 0.681 | Val Loss: 0.67

### Serialise / Deserialise Embedding Parameters after training

In [ ]:
params_after = params(classifier.emb)

In [ ]:
# Serialise embedding params after training
serialise('tensor', serialise_directory, 'params_after', params_after)

In [ ]:
# Deserialise embedding params after training
params_after = deserialise('tensor', serialise_directory, 'params_after')

In [ ]:
# check changes in params
params_after == params_before

In [ ]:
# check gradients
classifier.emb.weight.grad

In [ ]:
# trainable / all params after training
trainable_params, all_params = num_params(classifier)
print('The model has {:,} trainable parameters'.format(trainable_params))
print('The model has {:,} parameters overall'.format(all_params))

## Evaluating on test data

To evaluate the data on the held-out test set, the code is exactly the same as the validation loop in the training routine we saw in the previous step. 

The test set should be run as little as possible. Each time you run a trained model on the test set, make a new model decision (such as changing the size of the layers), and remeasure the new retrained model on the test set, you are biasing your
modeling decisions toward the test data. In other words, if you repeat that process often enough, the test set will become meaningless as an accurate measure of truly held-out data.

In [ ]:
classifier.eval()  

test_loss = 0
test_acc = 0
    
with torch.no_grad():     #torch.no_grad() deals with the autograd engine and stops it from calculating the gradients, which is the recommended way of doing validation
    for i, (x_test, y_test) in enumerate(test_loader):
        x_test = x_test.to(device) 
        y_test = y_test.to(device)
        
        # Forward propagation: compute the model output (i.e. predictions)
        y_pred = classifier(x_in=x_test)     #tensors of probabilities
        
        # Loss calculation
        tst_loss = loss_func(y_pred, y_test)
        
        # Accuracy
        tst_acc = binary_acc(y_pred, y_test)
            
        test_loss += tst_loss.item()
        test_acc += tst_acc.item()
            
print('Test Loss: {:.3f} | Test Acc: {:.3f}'.format(test_loss/len(test_loader), test_acc/len(test_loader)))

print()
print('Done!')

## Inference on New Data

In [ ]:
reviews = ["Suspended my account without warning and my order got delayed. I called them two days in a row to be fobbed off \
          and to wait for an email. No email. No access to my online account. 3 amazon prime payments taken from my card \
          without my permission. Just got access back to claim my money back 12 weeks later and didn't even get an apology \
          from the online chat. Didn't even get my order either.",
          'Fantastic!!!!',
          'The products they sent me always work fine but... I have Amazon prime and they sometimes deliver days late or \
          just leave the product on your doorstep out in the open easy to be stolen. Overall, my experience with Amazon is \
          fairly good',
          'Safe place never used…doorstep deliveries. I am a long standing Amazon customer and high spending. I have had \
          a safe place named for deliveries for a long time. The odd driver hitters to open the door to the bin shed and \
          out the parcels in. The majority just leave it on the doorstep. With thefts high, I find this utterly ridiculous. \
          What is the point of a safe place if it is simply ignored and never used? I live in a house with a small front \
          garden - not a difficult to access apartment block or something. Amazon - please hold your drivers accountable \
          to the requests of customers. You offer and unique and great service but this lets you down.',
          'My beautiful Jenuh killed by incompetence! She went to Dewey Veterinary Medical Center 3x! My MISTAKE!! I was \
          told she had nothing but allergies! She was filled with injections that DID NOTHING for her along with \
          medications that were ridiculously priced and again that did nothing!! 5 days after her last visit with Dewey \
          I’m in another veterinary office with her in critical condition and them saying she was on oxygen and had a very \
          bad infection! They’re telling me the rash on her body was NOT allergies, but was in fact signs of trouble from \
          infection!!! She was never treated just given more shots and new medications and here she was suffering inside! \
          (All I was told was many pets were coming in suffering from allergies) My 8 month beautiful blue and green-eyed \
          Husky died all because Dewey Vet did NOTHING for her!! She lost her life and I didn’t get to enjoy loving her \
          for many year’s because her symptoms were not taken seriously ! She didn’t have to die SHE WAS MURDERED by lack \
          of care!!!!! As a pet mom you take your babies into veterinarians offices in hopes they will properly diagnose \
          and care for your kids basicaly!!! Jenuh was not cared for! This vet told me her impacted teeth (Adult teeth \
          came in but baby teeth didn’t fall out) wasn’t urgent she could get them out when they spayed her too which I \
          said I was told by other vet that I should wait between 1-1.5 years old which he blew off and said NOT TRUE! \
          (I read about the urgency to pull teeth online yet this was also blown off as nothing) His old school veterinary \
          services may actually be OUTDATED garbage! Maybe it’s time to retire!!!!',
          "It's a good search engine and info that shows up is mostly helpful, but lately the ads are flooding the face \
          page which is not always the best sites for what you are looking for.",
          "I bought google play credit.I have the virtual card.But they blocked all my account transaction, when i \
          contacted them,they told me that they would solve it,Instead, they locked it.Even after submitting all proper \
          documents, they are refusing to reactive.Such bad experience", 
          'I have had a long term rental association (as a renter) with my Property Manager Kiera Hannaford. I would like \
          to acknowledge her professional, friendly and thorough knowledge of all things related to rental property \
          management. Kiera always does her very best to help and goes to a great deal of trouble to ensure a positive \
          outcome with all queries and requests. Her honesty is such a valuable asset when there are challenging times. \
          As a renter I have always been able to rely on Kiera for help. I wish her the very best for a very bright \
          future and would recommend her if you ever need an excellent team member in the real estate industry. ',
          "It’s short from being the best because I’m concerned over some huge privacy details that may get leaked to the \
          greater public but otherwise it’s a good engine.",
          "Always had good experience with google before but lately there's been a lot of ad popping up and not \
          necessarily nice sites just sites that paid money to google for google to put their sites on first page. \
          But google is a nice service with a lot of functions although I wonder how much personal information they \
          store of you in the database.",
          "Kindred Healthcare physical therapists (PT) and occupational therapists (OT) in Okaloosa County, Florida, \
          helped me regain use of my right arm and increase my stamina and endurance despite COPD problems. During my \
          in-patient and outpatient therapy, they were always professional and polite while encouraging me to challenge \
          myself. PT and OT helped me regain independence, self-reliance, and dignity. I am grateful.",
          "Had a problem. Resolved immediately. Great customer service!!!",
          "had 3 hour drive ahead of me and Adam the fitter could see my distress at having to wait he jacked up the truck \
          in a flash and had me on my way, top sevices from Adam and the kwikfit team.",
          "Online spas- Absolutely lovely guys and excellent customer service. The guys showed me how to set up and did \
          all the hard work for me. Nothing was any problem :). Lovely the hot tub and would highly recommend to \
          friends ! Thanks again!",
          "Mr. Handyman is a great service. The staff isprofessional and all of the jobs get done. I have used Mr. \
          Handyman several times and I couldn’t be happier."]

In [ ]:
#inference_pipeline.tasks

In [ ]:
inferences = [inference_pipeline.run(r) for r in reviews]

In [ ]:
for i,inf in enumerate(inferences):
    print('{}. {} --- probability value {:.2%}'.format(i+1, inf[0], inf[1]))

In [ ]:
# 1.
# "Suspended my account without warning and my order got delayed. I called them two days in a row to be fobbed off \
# and to wait for an email. No email. No access to my online account. 3 amazon prime payments taken from my card \
# without my permission. Just got access back to claim my money back 12 weeks later and didn't even get an apology \
# from the online chat. Didn't even get my order either." 
# 1. Negative --- probability value 3.39%    --- CORRECT

# 2.
# 'Fantastic!!!!'  
# 2. Positive --- probability value 98.76%   --- CORRECT

# 3.
# 'The products they sent me always work fine but... I have Amazon prime and they sometimes deliver days late or \
# just leave the product on your doorstep out in the open easy to be stolen. Overall, my experience with Amazon is \
# fairly good' 
# 3. Positive --- probability value 66.64%   --- CORRECT

# 4.
# 'Safe place never used…doorstep deliveries. I am a long standing Amazon customer and high spending. I have had \
# a safe place named for deliveries for a long time. The odd driver hitters to open the door to the bin shed and \
# out the parcels in. The majority just leave it on the doorstep. With thefts high, I find this utterly ridiculous. \
# What is the point of a safe place if it is simply ignored and never used? I live in a house with a small front \
# garden - not a difficult to access apartment block or something. Amazon - please hold your drivers accountable \
# to the requests of customers. You offer and unique and great service but this lets you down.'
# 4. Negative --- probability value 11.10%    --- CORRECT 

# 5.
# 'My beautiful Jenuh killed by incompetence! She went to Dewey Veterinary Medical Center 3x! My MISTAKE!! I was \
# told she had nothing but allergies! She was filled with injections that DID NOTHING for her along with \
# medications that were ridiculously priced and again that did nothing!! 5 days after her last visit with Dewey \
# I’m in another veterinary office with her in critical condition and them saying she was on oxygen and had a very \
# bad infection! They’re telling me the rash on her body was NOT allergies, but was in fact signs of trouble from \
# infection!!! She was never treated just given more shots and new medications and here she was suffering inside! \
# (All I was told was many pets were coming in suffering from allergies) My 8 month beautiful blue and green-eyed \
# Husky died all because Dewey Vet did NOTHING for her!! She lost her life and I didn’t get to enjoy loving her \
# for many year’s because her symptoms were not taken seriously ! She didn’t have to die SHE WAS MURDERED by lack \
# of care!!!!! As a pet mom you take your babies into veterinarians offices in hopes they will properly diagnose \
# and care for your kids basicaly!!! Jenuh was not cared for! This vet told me her impacted teeth (Adult teeth \
# came in but baby teeth didn’t fall out) wasn’t urgent she could get them out when they spayed her too which I \
# said I was told by other vet that I should wait between 1-1.5 years old which he blew off and said NOT TRUE! \
# (I read about the urgency to pull teeth online yet this was also blown off as nothing) His old school veterinary \
# services may actually be OUTDATED garbage! Maybe it’s time to retire!!!!'
# 5. Negative --- probability value 4.70%  --- CORRECT

# 6.
# "It's a good search engine and info that shows up is mostly helpful, but lately the ads are flooding the face \
# page which is not always the best sites for what you are looking for."
# 6. Positive --- probability value 93.89%  --- CORRECT 

#7.
# "I bought google play credit.I have the virtual card.But they blocked all my account transaction, when i \
# contacted them,they told me that they would solve it,Instead, they locked it.Even after submitting all proper \
# documents, they are refusing to reactive.Such bad experience"
# 7. Negative --- probability value 0.35%   --- CORRECT

# 8.
# 'I have had a long term rental association (as a renter) with my Property Manager Kiera Hannaford. I would like \
# to acknowledge her professional, friendly and thorough knowledge of all things related to rental property \
# management. Kiera always does her very best to help and goes to a great deal of trouble to ensure a positive \
# outcome with all queries and requests. Her honesty is such a valuable asset when there are challenging times. \
# As a renter I have always been able to rely on Kiera for help. I wish her the very best for a very bright \
# future and would recommend her if you ever need an excellent team member in the real estate industry. '
# 8. Positive --- probability value 97.86%  --- CORRECT

# 9.
# "It’s short from being the best because I’m concerned over some huge privacy details that may get leaked to the \
# greater public but otherwise it’s a good engine."
# 9. Positive --- probability value 93.98% --- CORRECT

# 10.
# "Always had good experience with google before but lately there's been a lot of ad popping up and not \
# necessarily nice sites just sites that paid money to google for google to put their sites on first page. \
# But google is a nice service with a lot of functions although I wonder how much personal information they \
# store of you in the database."
# 10. Positive --- probability value 79.14%   --- CORRECT

# 11.
# "Kindred Healthcare physical therapists (PT) and occupational therapists (OT) in Okaloosa County, Florida, \
# helped me regain use of my right arm and increase my stamina and endurance despite COPD problems. During my \
# in-patient and outpatient therapy, they were always professional and polite while encouraging me to challenge \
# myself. PT and OT helped me regain independence, self-reliance, and dignity. I am grateful."
# 11. Positive --- probability value 94.97%   --- CORRECT

# 12.
# "Had a problem. Resolved immediately. Great customer service!!!"
# 12. Positive --- probability value 79.75%   --- CORRECT 

# 13.
# "had 3 hour drive ahead of me and Adam the fitter could see my distress at having to wait he jacked up the truck \
# in a flash and had me on my way, top sevices from Adam and the kwikfit team."
# 13. Positive --- probability value 64.67% --- CORRECT

# 14.
# "Online spas- Absolutely lovely guys and excellent customer service. The guys showed me how to set up and did \
# all the hard work for me. Nothing was any problem :). Lovely the hot tub and would highly recommend to \
# friends ! Thanks again!"
# 14. Positive --- probability value 99.78%  --- CORRECT

# 15.
# "Mr. Handyman is a great service. The staff isprofessional and all of the jobs get done. I have used Mr. \
# Handyman several times and I couldn’t be happier."
# 15. Positive --- probability value 75.25% --- CORRECT